## Train MRCNN Model from pipeline
 


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os, sys, math, io, time
import gc
import numpy as np
import argparse
import platform
import tensorflow as tf
import keras
import keras.backend as KB
import platform
sys.path.append('../')

import mrcnn.model_mrcnn  as mrcnn_modellib
import mrcnn.visualize    as visualize
import mrcnn.new_shapes   as shapes
from datetime import datetime   

from mrcnn.config       import Config
from mrcnn.dataset      import Dataset 
from mrcnn.utils        import log, stack_tensors, stack_tensors_3d, write_stdout
from mrcnn.datagen      import data_generator, load_image_gt
from mrcnn.callbacks    import get_layer_output_1,get_layer_output_2
from mrcnn.coco         import CocoDataset, CocoConfig, CocoInferenceConfig, evaluate_coco, build_coco_results
# from mrcnn.visualize    import plot_gaussian
from mrcnn.prep_notebook import mrcnn_coco_train

import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)
start_time = datetime.now().strftime("%m-%d-%Y @ %H:%M:%S")
print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

##-----------------------------------------------

Using TensorFlow backend.


 Initialize CocoConfig object - super
 Linx  Linux
Tensorflow Version: 1.8.0   Keras Version : 2.1.6 

--> Execution started at: 10-10-2018 @ 19:02:13
    Tensorflow Version: 1.8.0   Keras Version : 2.1.6 


### Setup argparse parameters

In [2]:
##------------------------------------------------------------------------------------
## Parse command line arguments
##  
## Example:
##           train-shapes_gpu --epochs 12 --steps-in-epoch 7 --last_epoch 1234 --logs_dir mrcnn_logs
##------------------------------------------------------------------------------------
parser = argparse.ArgumentParser(description='Train Mask R-CNN on MS COCO.')
# parser.add_argument("command",
                    # metavar="<command>",
                    # help="'train' or 'evaluate' on MS COCO")
# parser.add_argument('--dataset', required=True,
                    # metavar="/path/to/coco/",
                    # help='Directory of the MS-COCO dataset')
# parser.add_argument('--limit', required=False,
                    # default=500,
                    # metavar="<image count>",
                    # help='Images to use for evaluation (defaults=500)')
                    
parser.add_argument('--model', required=False,
                    default='last',
                    metavar="/path/to/weights.h5",
                    help="MRCNN model weights file: 'coco' , 'init' , or Path to weights .h5 file ")

parser.add_argument('--fcn_model', required=False,
                    default='last',
                    metavar="/path/to/weights.h5",
                    help="FCN model weights file: 'init' , or Path to weights .h5 file ")

parser.add_argument('--logs_dir', required=True,
                    default='mrcnn_logs',
                    metavar="/path/to/logs/",
                    help='Logs and checkpoints directory (default=logs/)')

parser.add_argument('--last_epoch', required=False,
                    default=0,
                    metavar="<last epoch ran>",
                    help='Identify last completed epcoh for tensorboard continuation')
                    
parser.add_argument('--epochs', required=False,
                    default=1,
                    metavar="<epochs to run>",
                    help='Number of epochs to run (default=1)')
                    
parser.add_argument('--steps_in_epoch', required=False,
                    default=1,
                    metavar="<steps in each epoch>",
                    help='Number of batches to run in each epochs (default=5)')

parser.add_argument('--val_steps', required=False,
                    default=1,
                    metavar="<val steps in each epoch>",
                    help='Number of validation batches to run at end of each epoch (default=1)')
                    
parser.add_argument('--batch_size', required=False,
                    default=5,
                    metavar="<batch size>",
                    help='Number of data samples in each batch (default=5)')                    

parser.add_argument('--lr', required=False,
                    default=0.001,
                    metavar="<learning rate>",
                    help='Learning Rate (default=0.001)')
# args = parser.parse_args()


_StoreAction(option_strings=['--lr'], dest='lr', nargs=None, const=None, default=0.001, type=None, choices=None, help='Learning Rate (default=0.001)', metavar='<learning rate>')

####  Pass input parameters to argparse

In [3]:
# args = parser.parse_args("--epochs 100 --steps_in_epoch 128  --last_epoch 1264 --batch_size 8  --lr 0.5               --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model init".split())
# input_parms = "--epochs 100 --steps_in_epoch 100  --last_epoch 1264 --batch_size 25 --lr 0.8 --val_steps 5 --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
# input_parms +=" --model     /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 "

input_parms = "--epochs 1 --steps_in_epoch 10  --last_epoch 0 --batch_size 2 --lr 0.001 --val_steps 50 " 
input_parms +="--logs_dir train_mrcnn_coco "
input_parms +="--model coco"
input_parms +=" --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
print(input_parms)

args = parser.parse_args(input_parms.split())

--epochs 1 --steps_in_epoch 10  --last_epoch 0 --batch_size 2 --lr 0.001 --val_steps 50 --logs_dir train_mrcnn_coco --model coco --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5


In [4]:
##----------------------------------------------------------------------------------------------
## if debug is true set stdout destination to stringIO
##----------------------------------------------------------------------------------------------            
# debug = False
# if debug:
#     sys.stdout = io.StringIO()

In [5]:
print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

# print("Dataset: ", args.dataset)
# print("Logs:    ", args.logs)
# print("Limit:   ", args.limit)

print("    MRCNN Model        : ", args.model)
print("    FCN Model          : ", args.fcn_model)
print("    Log Dir            : ", args.logs_dir)
print("    Last Epoch         : ", args.last_epoch)
print("    Epochs to run      : ", args.epochs)
print("    Steps in each epoch: ", args.steps_in_epoch)
print("    Validation steps   : ", args.val_steps)
print("    Batch Size         : ", args.batch_size)




--> Execution started at: 10-10-2018 @ 19:02:13
    Tensorflow Version: 1.8.0   Keras Version : 2.1.6 
    MRCNN Model        :  coco
    FCN Model          :  /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5
    Log Dir            :  train_mrcnn_coco
    Last Epoch         :  0
    Epochs to run      :  1
    Steps in each epoch:  10
    Validation steps   :  50
    Batch Size         :  2


###  Setup Project Directories

In [6]:
##------------------------------------------------------------------------------------
## setup project directories
#---------------------------------------------------------------------------------
# # Root directory of the project 
# MODEL_DIR    :    Directory to save logs and trained model
# COCO_MODEL_PATH  : Path to COCO trained weights
#---------------------------------------------------------------------------------
syst = platform.system()
print("    OS Platform        : ", syst)
if syst == 'Windows':
    # Root directory of the project
    # WINDOWS MACHINE ------------------------------------------------------------------
    ROOT_DIR          = "E:\\"
    TRAINING_DIR   = os.path.join(ROOT_DIR, "models")
    DATASET_DIR    = os.path.join(ROOT_DIR, 'MLDatasets')
    PRETRAINED_DIR = os.path.join(ROOT_DIR, 'PretrainedModels')
elif syst == 'Linux':
    # LINUX MACHINE ------------------------------------------------------------------
    ROOT_DIR       = os.getcwd()
    TRAINING_DIR   = os.path.expanduser('~/models')
    DATASET_DIR    = os.path.expanduser('~/MLDatasets')
    PRETRAINED_DIR = os.path.expanduser('~/PretrainedModels')
else :
    raise Error('unreconized system  '      )

# MODEL_DIR    = os.path.join(TRAINING_DIR, "mrcnn_logs")

TRAINING_PATH     = os.path.join(TRAINING_DIR  , "train_mrcnn_coco")
COCO_DATASET_PATH = os.path.join(DATASET_DIR   , "coco2014")
COCO_MODEL_PATH   = os.path.join(PRETRAINED_DIR, "mask_rcnn_coco.h5")
RESNET_MODEL_PATH = os.path.join(PRETRAINED_DIR, "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")
VGG16_MODEL_PATH  = os.path.join(PRETRAINED_DIR, "fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5")

    OS Platform        :  Linux


### Build Configuration Object

In [7]:

##------------------------------------------------------------------------------------
## Build configuration object 
##------------------------------------------------------------------------------------
mrcnn_config                    = CocoConfig()
mrcnn_config.NAME               = 'mrcnn'              
mrcnn_config.TRAINING_PATH      = TRAINING_PATH
mrcnn_config.COCO_DATASET_PATH  = COCO_DATASET_PATH 
mrcnn_config.COCO_MODEL_PATH    = COCO_MODEL_PATH   
mrcnn_config.RESNET_MODEL_PATH  = RESNET_MODEL_PATH 
mrcnn_config.VGG16_MODEL_PATH   = VGG16_MODEL_PATH  
mrcnn_config.COCO_CLASSES       = None 
mrcnn_config.DETECTION_PER_CLASS = 200
mrcnn_config.HEATMAP_SCALE_FACTOR = 4

mrcnn_config.BATCH_SIZE         = int(args.batch_size)                  # Batch size is 2 (# GPUs * images/GPU).
mrcnn_config.IMAGES_PER_GPU     = int(args.batch_size)                  # Must match BATCH_SIZE
mrcnn_config.STEPS_PER_EPOCH    = int(args.steps_in_epoch)
mrcnn_config.LEARNING_RATE      = float(args.lr)
mrcnn_config.EPOCHS_TO_RUN      = int(args.epochs)
mrcnn_config.FCN_INPUT_SHAPE    = mrcnn_config.IMAGE_SHAPE[0:2]
mrcnn_config.LAST_EPOCH_RAN     = int(args.last_epoch)
mrcnn_config.WEIGHT_DECAY       = 2.0e-4
mrcnn_config.VALIDATION_STEPS   = int(args.val_steps)
mrcnn_config.REDUCE_LR_FACTOR   = 0.5
mrcnn_config.REDUCE_LR_COOLDOWN = 30
mrcnn_config.REDUCE_LR_PATIENCE = 40
mrcnn_config.EARLY_STOP_PATIENCE= 80
mrcnn_config.EARLY_STOP_MIN_DELTA = 1.0e-4
mrcnn_config.MIN_LR             = 1.0e-10
mrcnn_config.NEW_LOG_FOLDER     = True  


mrcnn_config.display() 


Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_CLASSES                   None
COCO_DATASET_PATH              /home/kbardool/MLDatasets/coco2014
COCO_MODEL_PATH                /home/kbardool/PretrainedModels/mask_rcnn_coco.h5
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DETECTION_PER_CLASS            200
EARLY_STOP_MIN_DELTA           0.0001
EARLY_STOP_PATIENCE            80
EPOCHS_TO_RUN                  1
FCN_INPUT_SHAPE                [1024 1024]
GPU_COUNT                      1
HEATMAP_SCALE_FACTOR           4
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LAS

In [8]:
# print(' Checkpoint directory  : ', mrcnn_config.TRAINING_DIR)
print(' Checkpoint folder     : ', mrcnn_config.TRAINING_PATH)
print(' COCO   Dataset Path   : ', mrcnn_config.COCO_DATASET_PATH)
print(' COCO   Model Path     : ', mrcnn_config.COCO_MODEL_PATH)
print(' ResNet Model Path     : ', mrcnn_config.RESNET_MODEL_PATH)
print(' VGG16  Model Path     : ', mrcnn_config.VGG16_MODEL_PATH)
    

 Checkpoint folder     :  /home/kbardool/models/train_mrcnn_coco
 COCO   Dataset Path   :  /home/kbardool/MLDatasets/coco2014
 COCO   Model Path     :  /home/kbardool/PretrainedModels/mask_rcnn_coco.h5
 ResNet Model Path     :  /home/kbardool/PretrainedModels/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
 VGG16  Model Path     :  /home/kbardool/PretrainedModels/fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5


###  Build Model

In [9]:
# mrcnn_model, dataset_train, test_generator, dataset_val, val_generator, mrcnn_config = mrcnn_coco_train(mode = 'training', mrcnn_config = mrcnn_config)
mrcnn_model, dataset_train, dataset_val, _ , _, mrcnn_config = mrcnn_coco_train(mode = 'training', mrcnn_config = mrcnn_config)

>>> Initialize ModelBase model 
   Mode:  training
   Model dir :  /home/kbardool/models/train_mrcnn_coco
>>> ModelBase initialiation complete
>>> ---Initialize MRCNN model, mode:  training

----------------------------
>>> Resnet Graph 
----------------------------
     Input_image shape : (?, 1024, 1024, 3)
     After ZeroPadding2D  : (?, 1030, 1030, 3) (?, 1030, 1030, 3)
     After Conv2D padding : (?, 512, 512, 64) (?, 512, 512, 64)
     After BatchNorm      : (?, 512, 512, 64) (?, 512, 512, 64)
     C1 Shape: (?, 256, 256, 64) (?, 256, 256, 64)
     C2 Shape:  (?, 256, 256, 256) (?, 256, 256, 256)
     C3 Shape:  (?, 128, 128, 512) (?, 128, 128, 512)
     C4 Shape:  (?, 64, 64, 1024) (?, 64, 64, 1024)
     C5 Shape:  (?, 32, 32, 2048) (?, 32, 32, 2048)

>>> Feature Pyramid Network (FPN) Graph 
     FPN P2 shape : (None, 256, 256, 256)
     FPN P3 shape : (None, 128, 128, 256)
     FPN P4 shape : (None, 64, 64, 256)
     FPN P5 shape : (None, 32, 32, 256)
     FPN P6 shape : (None,

    gt_scores_exp          :  (2, ?, 1)
    gt_array shape         :  (2, 200, 8) (2, 200, 8)
    scatter_ind shape      :  (2, 200, 3) (2, 200, 3)
    tf.shape(gt_array)[-1] :  8 (2, 200, 8)
    gt_scatter shape       :  (2, 81, 200, 8) (2, 81, 200, 8)
    sort_inds              :  <class 'tensorflow.python.framework.ops.Tensor'>  shape  (2, 81, 200)
    class_grid             :  <class 'tensorflow.python.framework.ops.Tensor'>  shape  (2, 81, 200)
    batch_grid             :  <class 'tensorflow.python.framework.ops.Tensor'>  shape  (2, 81, 200)
    gather_inds            :  (2, 81, 200, 3)
    gt_tensor.shape        :  (2, 81, 200, 8) (2, 81, 200, 8)

 
  > NEW build_heatmap() for  ['gt_heatmap']
    in_tensor shape        :  (2, 81, 200, 8)
    num bboxes per class   :  200
    heatmap scale        :  4 Dimensions:  w: 256  h: 256
    pt2_sum shape  :  (2, 81, 200)
    pt2_ind shape  :  (?, 3)
    pt2_dense shape:  (?, 8)
    X/Y shapes : (256, 256) (256, 256)
    Ones:     (?, 1, 

### Load saved weights

In [10]:
##------------------------------------------------------------------------------------
## Load Mask RCNN Model Weight file
##------------------------------------------------------------------------------------
# exclude_list = ["mrcnn_class_logits"]
#load_model(model, init_with = args.model)   
exclude_list = []
mrcnn_model.load_model_weights(init_with = args.model, exclude = exclude_list)   

print('==========================================')
print(" MRCNN MODEL Load weight file COMPLETE    ")
print('==========================================')

-----------------------------------------------
 Load Model with init parm: [ coco ]
 Exclude layers: 
-----------------------------------------------
 ---> coco
   >>> load_weights() from : /home/kbardool/PretrainedModels/mask_rcnn_coco.h5
{'_id': <h5py.h5f.FileID object at 0x7f47cac916d0>, '_swmr_mode': False}
 List Model layers and matching hd5 layers, if present :     
    0  input_image...................  1 
    1  zero_padding2d_1..............  1 
    2  conv1.........................  1 
    3  bn_conv1......................  1 
    4  activation_1..................  1 
    5  max_pooling2d_1...............  1 
    6  res2a_branch2a................  1 
    7  bn2a_branch2a.................  1 
    8  activation_2..................  1 
    9  res2a_branch2b................  1 
   10  bn2a_branch2b.................  1 
   11  activation_3..................  1 
   12  res2a_branch2c................  1 
   13  res2a_branch1.................  1 
   14  bn2a_branch2c................

                                0 bn3d_branch2c/gamma:0                Weights match (512,)
                                1 bn3d_branch2c/beta:0                 Weights match (512,)
                                2 bn3d_branch2c/moving_mean:0          Weights match (512,)
                                3 bn3d_branch2c/moving_variance:0      Weights match (512,)

 78 add_7                     ModelLayers : [('add_7', <keras.layers.merge.Add object at 0x7f47cdd2d1d0>)] 
                              Weights     : []

 79 res3d_out                 ModelLayers : [('res3d_out', <keras.layers.core.Activation object at 0x7f47cdd46908>)] 
                              Weights     : []

 80 res4a_branch2a            ModelLayers : [('res4a_branch2a', <keras.layers.convolutional.Conv2D object at 0x7f47cdcd3f60>)] 
                              Weights     : ['res4a_branch2a/kernel:0', 'res4a_branch2a/bias:0']
                                0 res4a_branch2a/kernel:0              Weights match

                                0 res4m_branch2c/kernel:0              Weights match (1, 1, 256, 1024)
                                1 res4m_branch2c/bias:0                Weights match (1024,)

209 bn4m_branch2c             ModelLayers : [('bn4m_branch2c', <mrcnn.batchnorm_layer.BatchNorm object at 0x7f47cd1f4e48>)] 
                              Weights     : ['bn4m_branch2c/gamma:0', 'bn4m_branch2c/beta:0', 'bn4m_branch2c/moving_mean:0', 'bn4m_branch2c/moving_variance:0']
                                0 bn4m_branch2c/gamma:0                Weights match (1024,)
                                1 bn4m_branch2c/beta:0                 Weights match (1024,)
                                2 bn4m_branch2c/moving_mean:0          Weights match (1024,)
                                3 bn4m_branch2c/moving_variance:0      Weights match (1024,)

210 add_20                    ModelLayers : [('add_20', <keras.layers.merge.Add object at 0x7f47cd193da0>)] 
                              Weight

                              Weights     : []

333 res5b_out                 ModelLayers : [('res5b_out', <keras.layers.core.Activation object at 0x7f47cc6e5978>)] 
                              Weights     : []

334 res5c_branch2a            ModelLayers : [('res5c_branch2a', <keras.layers.convolutional.Conv2D object at 0x7f47cc6f2d68>)] 
                              Weights     : ['res5c_branch2a/kernel:0', 'res5c_branch2a/bias:0']
                                0 res5c_branch2a/kernel:0              Weights match (1, 1, 2048, 512)
                                1 res5c_branch2a/bias:0                Weights match (512,)

335 bn5c_branch2a             ModelLayers : [('bn5c_branch2a', <mrcnn.batchnorm_layer.BatchNorm object at 0x7f47cc6f28d0>)] 
                              Weights     : ['bn5c_branch2a/gamma:0', 'bn5c_branch2a/beta:0', 'bn5c_branch2a/moving_mean:0', 'bn5c_branch2a/moving_variance:0']
                                0 bn5c_branch2a/gamma:0                Weights m

   Weights file loaded: /home/kbardool/PretrainedModels/mask_rcnn_coco.h5 
  set_log_dir(): self.Checkpoint_path: /home/kbardool/models/train_mrcnn_coco/mrcnn20181010T1902/mrcnn_{epoch:04d}.h5 
  set_log_dir(): self.log_dir        : /home/kbardool/models/train_mrcnn_coco/mrcnn20181010T1902 
  set_log_dir(): Last completed epoch (self.epoch): 0 
 load_model_weights() : MODEL Load weight file COMPLETE    
 MRCNN MODEL Load weight file COMPLETE    


In [11]:
mrcnn_config.display()  
mrcnn_model.layer_info()
print()
print("Testing Dataset Image Count: {}".format(len(dataset_train.image_ids)))
print("Testing Dataset Class Count: {}".format(dataset_train.num_classes))


Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_CLASSES                   None
COCO_DATASET_PATH              /home/kbardool/MLDatasets/coco2014
COCO_MODEL_PATH                /home/kbardool/PretrainedModels/mask_rcnn_coco.h5
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DETECTION_PER_CLASS            200
EARLY_STOP_MIN_DELTA           0.0001
EARLY_STOP_PATIENCE            80
EPOCHS_TO_RUN                  1
FCN_INPUT_SHAPE                [1024 1024]
GPU_COUNT                      1
HEATMAP_SCALE_FACTOR           4
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LAS

### Setup Optimization Method

In [12]:

##----------------------------------------------------------------------------------------------
## Setup optimizaion method 
##----------------------------------------------------------------------------------------------            
print('    learning rate : ', mrcnn_model.config.LEARNING_RATE)
print('    momentum      : ', mrcnn_model.config.LEARNING_MOMENTUM)
optimizer = keras.optimizers.SGD(lr=mrcnn_model.config.LEARNING_RATE, 
                                 momentum=mrcnn_model.config.LEARNING_MOMENTUM, clipnorm=5.0)
# optimizer= tf.train.GradientDescentOptimizer(learning_rate, momentum)
# optimizer = keras.optimizers.RMSprop(lr=learning_rate, rho=0.9, epsilon=None, decay=0.0)
# optimizer = keras.optimizers.Adagrad(lr=learning_rate, epsilon=None, decay=0.0)
# optimizer = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# optimizer = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)


    learning rate :  0.001
    momentum      :  0.9


## Train

In [13]:
mrcnn_model.config.EPOCHS_TO_RUN = 10
print(mrcnn_model.config.LAST_EPOCH_RAN)
print(mrcnn_model.config.LEARNING_RATE)
print(mrcnn_model.config.EPOCHS_TO_RUN)
print(mrcnn_model.config.WEIGHT_DECAY)
print(mrcnn_model.config.EPOCHS_TO_RUN)
print(mrcnn_model.config.STEPS_PER_EPOCH)

0
0.001
10
0.0002
10
10


In [14]:

##----------------------------------------------------------------------------------------------
## Train the head branches
## Passing layers="heads" freezes all layers except the head layers. 
## You can also pass a regular expression to select which layers to train by name pattern.
##----------------------------------------------------------------------------------------------

train_layers = [ 'mrcnn', 'fpn','rpn']   ## equivalent to "heads"
loss_names   = [ "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss"]
# train_layers = [ 'mrcnn']
# loss_names   = [ "mrcnn_class_loss", "mrcnn_bbox_loss"]

mrcnn_model.epoch = mrcnn_model.config.LAST_EPOCH_RAN
# mrcnn_model.config.LEARNING_RATE   = 0.001
# mrcnn_model.config.STEPS_PER_EPOCH = config.STEPS_PER_EPOCH

mrcnn_model.train(optimizer, dataset_train, dataset_val, 
            learning_rate = mrcnn_model.config.LEARNING_RATE, 
            epochs_to_run = mrcnn_model.config.EPOCHS_TO_RUN,
#             epochs = 25,            # total number of epochs to run (accross multiple trainings)
#             batch_size = 0
#             steps_per_epoch = 0 
            layers = train_layers,
            losses = loss_names)
            


['mrcnn', 'fpn', 'rpn']
['(mrcnn\\_.*)', '(fpn\\_.*)', '(rpn\\_.*)']
layers regex : (mrcnn\_.*)|(fpn\_.*)|(rpn\_.*)
type train_dataset: <class 'mrcnn.coco.CocoDataset'>
type val_dataset: <class 'mrcnn.coco.CocoDataset'>
 344  fpn_c5p5               (Conv2D              )   TRAIN 
 346  fpn_c4p4               (Conv2D              )   TRAIN 
 349  fpn_c3p3               (Conv2D              )   TRAIN 
 352  fpn_c2p2               (Conv2D              )   TRAIN 
 354  fpn_p5                 (Conv2D              )   TRAIN 
 355  fpn_p2                 (Conv2D              )   TRAIN 
 356  fpn_p3                 (Conv2D              )   TRAIN 
 357  fpn_p4                 (Conv2D              )   TRAIN 
       1  rpn_conv_shared        (Conv2D              )   TRAIN 
       2  rpn_class_raw          (Conv2D              )   TRAIN 
       4  rpn_bbox_pred          (Conv2D              )   TRAIN 
 368  mrcnn_class_conv1      (TimeDistributed     )   TRAIN 
 369  mrcnn_class_bn1        (TimeDi

/home/kbardool/anaconda3/envs/TFG/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10


/home/kbardool/anaconda3/envs/TFG/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


10/10 [==============================] - 1174s 117s/step - loss: 0.4115 - rpn_class_loss: 0.0089 - rpn_bbox_loss: 0.1657 - mrcnn_class_loss: 0.1070 - mrcnn_bbox_loss: 0.1299 - val_loss: 1.6421 - val_rpn_class_loss: 0.0839 - val_rpn_bbox_loss: 0.8983 - val_mrcnn_class_loss: 0.4516 - val_mrcnn_bbox_loss: 0.2083

Epoch 00001: val_loss improved from inf to 1.64213, saving model to /home/kbardool/models/train_mrcnn_coco/mrcnn20181010T1902/mrcnn_0001.h5
Epoch 2/10
 1/10 [==>...........................] - ETA: 15:42 - loss: 0.9333 - rpn_class_loss: 0.0299 - rpn_bbox_loss: 0.2652 - mrcnn_class_loss: 0.4369 - mrcnn_bbox_loss: 0.2013

KeyboardInterrupt: 

## Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_2500.h5")
model.keras_model.save_weights(model_path)